# GAT 图神经网络学习笔记

_关于图、GAT 的学习记录_

_为何叫 GAT 捏，因为 GAN 一般指的是 Generative Adversal Nets_

## 数据结构-图

图的三个特征

- `node` 节点，每个顶点有着自己的特征，node2vec 将顶点的特征转换为一个高维向量 $h_i$
- `edge` 边，节点之间的连接
- `global` 图，包含着全局特征

下面这个图就是一个无向连接的图 $G$ ，其有五个节点，每个节点有**相邻节点**和此节点自身的**特征（可以是一个数值、向量、矩阵）**  
![](https://pic1.zhimg.com/80/v2-ec415ca61d7eef27296aff1994e91db8_1440w.webp)

可写出图 G 的邻接矩阵 A ：

![](./img/邻接矩阵.png)

## inductive 和 transductive

[如何理解 inductive learning 与 transductive learning?](https://www.zhihu.com/question/68275921)  
区别在于**预测的样本**是否在我们训练的时候已经用过  
`transductive` 直推式学习 训练阶段和测试阶段都基于同样的图结构  
`inductive` 归纳式学习 处理动态图 训练阶段和测试阶段要处理的图不同


## GAT 架构

**Graph Attention Network**

有着两种计算方式  
`Global Graph Attention` 节点 i 与图上所有节点都做 attention 计算  
`Mask Graph Attention` 节点 i 只与邻居节点做 attention 计算， GAT 使用的是这个方法

### Graph Attentional Layer

GAT 架构通过堆叠图注意力层来实现  
先来看注意力系数(attention coefficients)的计算：  
$$e_{ij} = a(W\vec{h_i}, W\vec{h_j})$$  
eij 表示节点 j 的特征对节点 i 的影响

下面为 GAL 的核心公式：  
$$a_{ij} = softmax(\sigma(\vec{a}^T[W\vec{h_i}||W\vec{h_j}]))$$
_其中 || 为 concatenate，表示张量的粘合，比如[[1， 2], [3，4]]粘合[[5, 6], [7, 8]]变成[[1， 2], [3，4]，[5, 6], [7, 8]]_  
$\vec{h_i}$和$\vec{h_j}$表示张量的节点 i 和 j 的节点特征，维度是 1 x $F$  
$W$是权重矩阵，维度$F'$ x $F$ ，这个权重矩阵是共享的，可以应用于每一个节点  
$\vec{a}^T$ 为 attention kernal, 维度为$2F'$ x 1  
$\sigma$ 激活函数用的是 LeakyReLu(负斜率=0.2)  
关于激活函数可以阅读[激活函数(Sigmoid/ReLU/LeakyReLU/PReLU/ELU)](https://zhuanlan.zhihu.com/p/172254089)这篇文章  
观察这些维度，会惊奇的发现经过公式的一顿操作之后的结果是一个实数 R, 这个数就是 attention 系数，表明 j 节点对 i 节点的重要程度  
这时候再看论文里的这个图系不系了然于心 😉

![](./img/图注意力机制.png)

了解了$a_{ij}$的计算公式，再来看这个公式  
$$ \vec{h*i}' = \sigma(\sum*{j\in{N*i}}a*{ij}W\vec{h*j})$$
$\vec{h_i}'$ 表示这层 GAL 关于节点 i 的输出特征  
$N_i$ 表示节点 i 的邻接节点  
$a*{ij}$ 表示上述公式的结果即注意力系数
这里的$\sigma$激活函数采用的是 ELU

### multi-head attention

先看这个图

![](./img/多头注意力.png)

图上 3 条不同颜色的线就代表着 3 个独立的 attention 系数
$$\vec{h_i}' = \parallel_{K=1}^K\sigma(\sum_{j\in{N_i}}a_{ij}^kW^k\vec{h_j})$$
上面公式代表中间层的输出形式
最后一层即预测层采取加权平方法，下面的公式为输出层形式：
$$\vec{h_i}' = \sigma(\frac{1}{K}\sum_{k=1}^K\sum_{j\in{N_i}}a_{ij}^kW^k\vec{h_j})$$
输出层的$\sigma$用的是 softmax


## GAT 的优点

1. 计算速度快， 可以在不同节点上进行并行运算
2. 可以同时对拥有不同度的节点进行处理
3. 可以被直接用于解决归纳学习(inductive)问题，即可以对从未见过的图结构进行处理


---

## 参考

[【GNN】图注意力网络 GAT](https://zhuanlan.zhihu.com/p/112938037)


## 英语学习

### 专业类词汇

inductive 归纳  
transductive 转导  
benchmark 基准  
recursive 递归的、循环的  
grid-like 网格状  
acyclic 非周期、非环状  
cyclic  
sepctral  
aggregated 聚集  
explicit 显式的  
coefficient 系数

### 描述类

leverage 使用、利用  
arbitrarily  
intense computation 密集计算
